## B. PASSING THE CONTEXT TO AN LLM 

In [4]:
#!pip install openai

In [3]:
from openai import OpenAI

# client = OpenAI(
#     base_url='http://localhost/v1',
#     api_key='ollama',
# )

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [5]:
#getting the documents to index 

import requests 


docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# sample document index from minsearch
documents[0]

#connecting to the Elastic Search client
#Elastic search saves all the index data on disk - may need to do volume mapping for more 
#advanced persisting.
from elasticsearch import Elasticsearch

In [9]:
es_client  = Elasticsearch('http://localhost:9200')

es_client.info()

#Creating an index in the database
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


In [13]:
resp = es_client.indices.delete( index="course-questions", ) 
print(resp)

{'acknowledged': True}


In [14]:
index_name = "course-questions"

es_client.indices.create(index=index_name , body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [15]:
from tqdm.auto import tqdm

In [16]:
#indexing the FAQ documents
for doc in documents:
    es_client.index(index=index_name, document = doc)

#querying the indexed documents 
# "question^3" - means that question is 3 times more important than the text and section fields
# size means we get five results back 



### Modularizing the code 

In [17]:
def elastic_search(query) : 
    
    query = "I just discovered the course , can I still join ?"
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    
    #collecting the several documents into one List (Constnat time complexity)
    result_docs = [] 
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [18]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [21]:
client.chat.completions.create(model='phi3',messages=[{"role": "user", "content": 'how are you?'}])

InternalServerError: Error code: 500 - {'error': {'message': 'model requires more system memory (5.6 GiB) than is available (4.3 GiB)', 'type': 'api_error', 'param': None, 'code': None}}

In [19]:
# modularized calls 
query = "how do I run kafka?"

def rag(query):
 
    search_results = elastic_search(query)
    
    prompt = build_prompt(query, search_results)
    
    answer = llm(prompt)

    return answer

In [20]:
rag('the course has already started , can I still enroll ?')

APIConnectionError: Connection error.

## D. PERSISTING DOCUMENT INDEXES USING ELASTICSEARCH FROM A DOCKER CONTAINER

In [13]:
# sample document index from minsearch
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [18]:
#connecting to the Elastic Search client
#Elastic search saves all the index data on disk - may need to do volume mapping for more 
#advanced persisting.
from elasticsearch import Elasticsearch

In [15]:
es_client  = Elasticsearch('http://localhost:9200')

In [16]:
es_client.info()

ObjectApiResponse({'name': 'acd1e8272a57', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0Ijp0t7TT-u0EENOYZ2h9g', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [19]:
#Creating an index in the database
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name , body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [20]:
from tqdm.auto import tqdm

In [21]:
#indexing the FAQ documents
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

In [28]:
#querying the indexed documents 
# "question^3" - means that question is 3 times more important than the text and section fields
# size means we get five results back 

def elastic_search(query) : 
    
    query = "I just discovered the course , can I still join ?"
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    
    #collecting the several documents into one List (Constnat time complexity)
    result_docs = [] 
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs




In [46]:
#elastic_search(query)

In [47]:
#Adjusting the RAG workflow

# modularized calls 
query = "'the course has already started , can I still enroll ?"

def rag(query):
 
    search_results = elastic_search(query)
    
    prompt = build_prompt(query, search_results)
    
    answer = llm(prompt)

    return answer



In [48]:
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [50]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00m eta 0:00:01


In [55]:
## E. Calaculating prompting costs 

In [52]:
import tiktoken

In [53]:
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

In [54]:
search_results = elastic_search(query)

prompt = build_prompt(query, search_results)

len(encoding.encode(prompt))

467

In [77]:
#gpt mini costs for sychronous API
input_cost = (0.150/1000000) * len(encoding.encode(prompt))
output_cost = (0.6/1000000 ) * len(encoding.encode(rag(query)))

In [79]:
total_cost = "{:.8f}".format(input_cost + output_cost)

In [80]:
tokens = encoding.encode(prompt)[:10]


tokens , print(f"The cost of this prompt is ${total_cost}".format('a'))

The cost of this prompt is $0.00009105


([3575, 553, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122], None)